In [1]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

/kaggle/input/protein-embeddings/fasta_embeddings_final.csv
/kaggle/input/cafa-6-protein-function-prediction/sample_submission.tsv
/kaggle/input/cafa-6-protein-function-prediction/IA.tsv
/kaggle/input/cafa-6-protein-function-prediction/Test/testsuperset.fasta
/kaggle/input/cafa-6-protein-function-prediction/Test/testsuperset-taxon-list.tsv
/kaggle/input/cafa-6-protein-function-prediction/Train/train_terms.tsv
/kaggle/input/cafa-6-protein-function-prediction/Train/train_sequences.fasta
/kaggle/input/cafa-6-protein-function-prediction/Train/train_taxonomy.tsv
/kaggle/input/cafa-6-protein-function-prediction/Train/go-basic.obo


In [2]:
# !pip install torch==2.7.0
!pip install transformers datasets sentence-transformers peft accelerate bitsandbytes torch
!pip install torch-geometric
!pip install biopython
!pip install obonet
!pip install networkx
!pip install pandas
!pip install numpy
!pip install matplotlib
!pip install seaborn
!pip install scipy
!pip install scikit-learn
!pip install fair-esm
!pip install pyarrow==19.0.0

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 59.4/59.4 MB 29.3 MB/s eta 0:00:00:00:0100:01
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 363.4/363.4 MB 5.0 MB/s eta 0:00:00:00:0100:01
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 13.8/13.8 MB 94.9 MB/s eta 0:00:00:00:0100:01
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 24.6/24.6 MB 81.3 MB/s eta 0:00:00:00:0100:01
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 883.7/883.7 kB 45.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 664.8/664.8 MB 2.5 MB/s eta 0:00:00:00:0100:01
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 211.5/211.5 MB 5.9 MB/s eta 0:00:00:00:0100:01
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 56.3/56.3 MB 33.8 MB/s eta 0:00:00:00:0100:01
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 127.9/127.9 MB 13.4 MB/s eta 0:00:0000:0100:01
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 207.5/207.5 MB 2.2 MB/s eta 0:00:00:00:0100:01
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 21.1/21.1 MB 60.2 MB/s eta 0:00:0000:0100:01
   ━━━━━━━━━━━━

In [3]:
import pandas as pd
import numpy as np
from sentence_transformers import SentenceTransformer
from datasets import Dataset
from transformers import (
    AutoModelForCausalLM,
    AutoTokenizer,
    TrainingArguments,
    Trainer,
    DataCollatorForLanguageModeling
)
from peft import LoraConfig, get_peft_model, prepare_model_for_kbit_training
import torch
from typing import List, Dict, Tuple
import json
import matplotlib.pyplot as plt
from sklearn.model_selection import train_test_split
import Bio
from Bio import SeqIO
import obonet
import gc
from sklearn.decomposition import PCA
from sklearn.metrics import confusion_matrix, ConfusionMatrixDisplay
from sklearn.preprocessing import LabelEncoder
import random
import esm

2025-12-10 00:05:55.402407: E external/local_xla/xla/stream_executor/cuda/cuda_fft.cc:477] Unable to register cuFFT factory: Attempting to register factory for plugin cuFFT when one has already been registered
E0000 00:00:1765325155.597820      47 cuda_dnn.cc:8310] Unable to register cuDNN factory: Attempting to register factory for plugin cuDNN when one has already been registered
E0000 00:00:1765325155.650258      47 cuda_blas.cc:1418] Unable to register cuBLAS factory: Attempting to register factory for plugin cuBLAS when one has already been registered


AttributeError: 'MessageFactory' object has no attribute 'GetPrototype'

AttributeError: 'MessageFactory' object has no attribute 'GetPrototype'

AttributeError: 'MessageFactory' object has no attribute 'GetPrototype'

AttributeError: 'MessageFactory' object has no attribute 'GetPrototype'

AttributeError: 'MessageFactory' object has no attribute 'GetPrototype'

In [4]:
obo_path = '/kaggle/input/cafa-6-protein-function-prediction/Train/go-basic.obo'
fasta_path = '/kaggle/input/cafa-6-protein-function-prediction/Train/train_sequences.fasta'
term_path = '/kaggle/input/cafa-6-protein-function-prediction/Train/train_terms.tsv'
embedding_path = '/kaggle/input/protein-embeddings/fasta_embeddings_final.csv'

In [5]:
def load_protein_embeddings(csv_path: str) -> pd.DataFrame:
    """Load protein embeddings from CSV file."""
    df = pd.read_csv(csv_path)
    print(f"Loaded {len(df)} protein embeddings")
    return df

def load_gene_ontologies(csv_path: str) -> pd.DataFrame:
    """Load gene ontology mappings."""
    df = pd.read_csv(csv_path, sep = '\t')
    print(f"Loaded {len(df)} GO annotations")
    return df

def encode_go_terms(go_df: pd.DataFrame) -> Dict[str, np.ndarray]:
    """
    Convert GO terms into embeddings using a sentence transformer.
    GO terms like 'GO:0008150' are converted to descriptive text.
    """
    print("Encoding GO terms...")
    
    # Initialize sentence transformer for GO term encoding
    model = SentenceTransformer('all-MiniLM-L6-v2')
    
    # Group by entry_id to get all GO terms per protein
    go_grouped = go_df.groupby('EntryID')['term'].apply(list).to_dict()
    
    go_embeddings = {}
    for entry_id, go_terms in go_grouped.items():
        # Convert GO terms to text (you can enhance this with GO descriptions)
        go_text = ", ".join(go_terms)
        embedding = model.encode(go_text)
        go_embeddings[entry_id] = embedding
    
    print(f"Encoded GO terms for {len(go_embeddings)} proteins")
    return go_embeddings

In [6]:
def create_qa_dataset(protein_df: pd.DataFrame, 
                     go_df: pd.DataFrame) -> List[Dict]:
    """
    Create question-answer pairs where:
    - Question: Protein embedding (converted to text representation)
    - Answer: Gene ontology terms
    """
    qa_pairs = []
    
    # Group GO terms by entry_id
    go_grouped = go_df.groupby('EntryID')['term'].apply(list).to_dict()
    
    for idx, row in protein_df.iterrows():
        entry_id = row['EntryID']
        
        if entry_id not in go_grouped:
            continue
        
        # Extract protein embedding (assuming columns after entry_id are embeddings)
        embedding_cols = [col for col in protein_df.columns if col != 'EntryID']
        protein_embedding = row[embedding_cols].values
        
        # Convert embedding to text representation (comma-separated values)
        embedding_str = ", ".join([f"{val:.4f}" for val in protein_embedding])  # First 10 dims
        
        # Get GO terms for this protein
        go_terms = go_grouped[entry_id]
        go_answer = "; ".join(go_terms)
        
        # Create QA pair
        qa_pair = {
            "question": f"What are the gene ontology annotations for a protein with embedding signature: [{embedding_str}...]?",
            "answer": f"The gene ontology terms are: {go_answer}",
            "entry_id": entry_id,
            "full_embedding": protein_embedding.tolist()
        }
        
        qa_pairs.append(qa_pair)
    
    print(f"Created {len(qa_pairs)} QA pairs")
    return qa_pairs

def format_for_training(qa_pairs: List[Dict], test_size: float = 0.2) -> Tuple[Dataset, Dataset]:
    """Format QA pairs for training and validation."""
    formatted_data = []
    
    for pair in qa_pairs:
        text = f"### Question: {pair['question']}\n### Answer: {pair['answer']}"
        formatted_data.append({"text": text})
    
    # Split into train and validation
    train_data, val_data = train_test_split(
        formatted_data, 
        test_size=test_size, 
        random_state=42
    )
    
    train_dataset = Dataset.from_list(train_data)
    val_dataset = Dataset.from_list(val_data)
    
    print(f"Train samples: {len(train_dataset)}, Validation samples: {len(val_dataset)}")
    
    return train_dataset, val_dataset

In [7]:
def setup_model_and_tokenizer(model_name: str = "microsoft/phi-2"):
    """
    Setup a lightweight model. Options:
    - microsoft/phi-2 (2.7B)
    - stabilityai/stablelm-2-1_6b
    - TinyLlama/TinyLlama-1.1B-Chat-v1.0
    """
    print(f"Loading model: {model_name}")
    
    tokenizer = AutoTokenizer.from_pretrained(model_name, trust_remote_code=True)
    tokenizer.pad_token = tokenizer.eos_token
    
    model = AutoModelForCausalLM.from_pretrained(
        model_name,
        torch_dtype=torch.float16,
        device_map="auto",
        trust_remote_code=True
    )
    
    return model, tokenizer

def tokenize_function(examples, tokenizer, max_length=512):
    """Tokenize the dataset."""
    return tokenizer(
        examples["text"],
        truncation=True,
        max_length=max_length,
        padding="max_length"
    )

def finetune_model(model, tokenizer, train_dataset, val_dataset, output_dir="./protein_go_model"):
    """Fine-tune the model with LoRA for efficiency."""
    
    # Configure LoRA
    lora_config = LoraConfig(
        r=8,  # Rank
        lora_alpha=16,
        target_modules=["q_proj", "v_proj"],  # Adjust based on model
        lora_dropout=0.05,
        bias="none",
        task_type="CAUSAL_LM"
    )
    
    # Apply LoRA
    model = prepare_model_for_kbit_training(model)
    model = get_peft_model(model, lora_config)
    
    print(f"Trainable parameters: {model.print_trainable_parameters()}")
    
    # Training arguments
    training_args = TrainingArguments(
        output_dir=output_dir,
        num_train_epochs=1,
        per_device_train_batch_size=4,
        per_device_eval_batch_size=4,
        gradient_accumulation_steps=4,
        learning_rate=2e-4,
        fp16=True,
        save_steps=1000,
        eval_steps=1000,  # Evaluate every 50 steps
        logging_steps=10,
        save_total_limit=2,
        warmup_steps=50,
        eval_strategy="steps",  # Enable evaluation
        load_best_model_at_end=True,
        metric_for_best_model="eval_loss",
        report_to="none"
    )
    
    # Data collator
    data_collator = DataCollatorForLanguageModeling(
        tokenizer=tokenizer,
        mlm=False
    )
    
    # Trainer
    trainer = Trainer(
        model=model,
        args=training_args,
        train_dataset=train_dataset,
        eval_dataset=val_dataset,  # Add validation dataset
        data_collator=data_collator
    )
    
    # Train
    print("Starting training...")
    train_result = trainer.train()
    
    # Save model
    model.save_pretrained(output_dir)
    tokenizer.save_pretrained(output_dir)
    print(f"Model saved to {output_dir}")
    
    return trainer

In [8]:
def plot_loss_curves(trainer, output_path="loss_curves.png"):
    """Plot training and validation loss curves."""
    log_history = trainer.state.log_history
    
    # Extract training and validation losses
    train_loss = []
    train_steps = []
    val_loss = []
    val_steps = []
    
    for entry in log_history:
        if 'loss' in entry:
            train_loss.append(entry['loss'])
            train_steps.append(entry['step'])
        if 'eval_loss' in entry:
            val_loss.append(entry['eval_loss'])
            val_steps.append(entry['step'])
    
    # Create figure with two subplots
    fig, (ax1, ax2) = plt.subplots(1, 2, figsize=(15, 5))
    
    # Plot 1: Combined view
    ax1.plot(train_steps, train_loss, 'b-', label='Training Loss', linewidth=2)
    ax1.plot(val_steps, val_loss, 'r-', label='Validation Loss', linewidth=2)
    ax1.set_xlabel('Steps', fontsize=12)
    ax1.set_ylabel('Loss', fontsize=12)
    ax1.set_title('Training and Validation Loss', fontsize=14, fontweight='bold')
    ax1.legend(fontsize=10)
    ax1.grid(True, alpha=0.3)
    
    # Plot 2: Separate subplots for better visibility
    ax2_twin = ax2.twinx()
    
    line1 = ax2.plot(train_steps, train_loss, 'b-', label='Training Loss', linewidth=2)
    line2 = ax2_twin.plot(val_steps, val_loss, 'r-', label='Validation Loss', linewidth=2)
    
    ax2.set_xlabel('Steps', fontsize=12)
    ax2.set_ylabel('Training Loss', fontsize=12, color='b')
    ax2_twin.set_ylabel('Validation Loss', fontsize=12, color='r')
    ax2.set_title('Loss Curves (Dual Axis)', fontsize=14, fontweight='bold')
    ax2.tick_params(axis='y', labelcolor='b')
    ax2_twin.tick_params(axis='y', labelcolor='r')
    ax2.grid(True, alpha=0.3)
    
    # Combine legends
    lines = line1 + line2
    labels = [l.get_label() for l in lines]
    ax2.legend(lines, labels, loc='upper right', fontsize=10)
    
    plt.tight_layout()
    plt.savefig(output_path, dpi=300, bbox_inches='tight')
    print(f"Loss curves saved to {output_path}")
    plt.show()
    
    # Print final metrics
    print("\n=== Final Metrics ===")
    print(f"Final Training Loss: {train_loss[-1]:.4f}")
    print(f"Final Validation Loss: {val_loss[-1]:.4f}")
    print(f"Best Validation Loss: {min(val_loss):.4f} at step {val_steps[val_loss.index(min(val_loss))]}")
    
    return train_loss, val_loss

def save_metrics_summary(train_loss, val_loss, output_path="metrics_summary.txt"):
    """Save a summary of training metrics."""
    with open(output_path, 'w') as f:
        f.write("=== Training Metrics Summary ===\n\n")
        f.write(f"Total training steps: {len(train_loss)}\n")
        f.write(f"Total validation evaluations: {len(val_loss)}\n\n")
        
        f.write(f"Initial Training Loss: {train_loss[0]:.4f}\n")
        f.write(f"Final Training Loss: {train_loss[-1]:.4f}\n")
        f.write(f"Training Loss Reduction: {((train_loss[0] - train_loss[-1]) / train_loss[0] * 100):.2f}%\n\n")
        
        f.write(f"Initial Validation Loss: {val_loss[0]:.4f}\n")
        f.write(f"Final Validation Loss: {val_loss[-1]:.4f}\n")
        f.write(f"Best Validation Loss: {min(val_loss):.4f}\n")
        f.write(f"Validation Loss Reduction: {((val_loss[0] - val_loss[-1]) / val_loss[0] * 100):.2f}%\n\n")
        
    
    print(f"Metrics summary saved to {output_path}")

In [ ]:
def main():
    # File paths (modify these)
    PROTEIN_EMBEDDING_CSV = embedding_path
    GO_ANNOTATIONS_CSV = term_path
    
    # Step 1: Load data
    print("\n=== Step 1: Loading Data ===")
    protein_df = load_protein_embeddings(PROTEIN_EMBEDDING_CSV)
    go_df = load_gene_ontologies(GO_ANNOTATIONS_CSV)
    print("Protein data")

    print(protein_df)

    print("Go data")

    print(go_df)
    
    # # Step 2: Create QA dataset
    print("\n=== Step 2: Creating QA Dataset ===")
    qa_pairs = create_qa_dataset(protein_df, go_df)
    # print("QA data")
    # print(qa_pairs)
    
    # # Save QA pairs for reference
    with open("qa_pairs.json", "w") as f:
        json.dump(qa_pairs, f, indent=2)
    
    # Step 3: Format for training
    print("\n=== Step 3: Formatting Dataset ===")
    train_dataset, val_dataset = format_for_training(qa_pairs, test_size=0.2)
    
    # Step 4: Setup model
    print("\n=== Step 4: Setting Up Model ===")
    model, tokenizer = setup_model_and_tokenizer("TinyLlama/TinyLlama-1.1B-Chat-v1.0")
    
    # # Step 5: Tokenize dataset
    print("\n=== Step 5: Tokenizing Dataset ===")
    train_dataset = train_dataset.map(
        lambda x: tokenize_function(x, tokenizer),
        batched=True,
        remove_columns=["text"]
    )
    val_dataset = val_dataset.map(
        lambda x: tokenize_function(x, tokenizer),
        batched=True,
        remove_columns=["text"]
    )
    
    # Step 6: Fine-tune
    print("\n=== Step 6: Fine-tuning Model ===")
    trainer = finetune_model(model, tokenizer, train_dataset, val_dataset)
    
    # Step 7: Plot loss curves
    print("\n=== Step 7: Plotting Loss Curves ===")
    train_loss, val_loss = plot_loss_curves(trainer)
    save_metrics_summary(train_loss, val_loss)
    
    print("\n=== Training Complete! ===")

# ===== INFERENCE EXAMPLE =====

# def inference_example(model_path="./protein_go_model"):
#     """Example of how to use the fine-tuned model."""
#     from transformers import pipeline
    
#     generator = pipeline(
#         "text-generation",
#         model=model_path,
#         device=0 if torch.cuda.is_available() else -1
#     )
    
#     sample_embedding = "[0.1234, 0.5678, -0.2341, ...]"
#     prompt = f"### Question: What are the gene ontology annotations for a protein with embedding signature: {sample_embedding}?\n### Answer:"
    
#     result = generator(prompt, max_length=200, num_return_sequences=1)
#     print(result[0]['generated_text'])

if __name__ == "__main__":
    # Install required packages first:
    # pip install transformers datasets sentence-transformers peft accelerate bitsandbytes
    
    main()


=== Step 1: Loading Data ===
Loaded 82154 protein embeddings
Loaded 537027 GO annotations
Protein data
      EntryID  prot_emb_0  prot_emb_1  prot_emb_2  prot_emb_3  prot_emb_4  \
0      P86164    0.063601    0.090173    0.272553    0.045446   -0.024562   
1      P84910    0.164426   -0.128475    0.244240    0.068856   -0.031558   
2      P83012    0.126772   -0.093400    0.216583    0.056300   -0.058426   
3      P83246    0.039001   -0.228608    0.207255    0.138270    0.081464   
4      P86133   -0.063519    0.141610    0.168906   -0.003960   -0.064855   
...       ...         ...         ...         ...         ...         ...   
82149  Q8WXI7   -0.078388    0.013259   -0.240767   -0.074411    0.519618   
82150  Q9I7U4   -0.198918   -0.103124   -0.017363    0.184486    0.247169   
82151  G4SLH0   -0.208552    0.005082    0.040566    0.072686    0.100601   
82152  Q8WZ42   -0.148885   -0.039154   -0.102000    0.173611    0.317519   
82153  A2ASS6   -0.142398   -0.053878   -0.089802

tokenizer_config.json: 0.00B [00:00, ?B/s]

tokenizer.model:   0%|          | 0.00/500k [00:00<?, ?B/s]

tokenizer.json: 0.00B [00:00, ?B/s]

special_tokens_map.json:   0%|          | 0.00/551 [00:00<?, ?B/s]

...

Map:   0%|          | 0/16431 [00:00<?, ? examples/s]


=== Step 6: Fine-tuning Model ===


No label_names provided for model class `PeftModelForCausalLM`. Since `PeftModel` hides base models input arguments, if label_names is not given, label_names can't be set automatically within `Trainer`. Note that empty label_names list will be used instead.


trainable params: 1,126,400 || all params: 1,101,174,784 || trainable%: 0.1023
Trainable parameters: None
Starting training...


Step,Training Loss,Validation Loss
1000,0.974600,0.985005


In [ ]:
df = pd.read_json("/kaggle/working/qa_pairs.json")
df.head()